In [ ]:
import awkward as ak
import dask
import dask_awkward
from distributed import performance_report
import hist
from hist.dask import Hist
import json
from coffea import processor
from coffea.nanoevents import BaseSchema, PFNanoAODSchema, NanoAODSchema
from coffea.dataset_tools import apply_to_dataset, apply_to_fileset, preprocess
from coffea.dataset_tools import max_chunks, max_files, slice_chunks, slice_files
from coffea.analysis_tools import Weights, PackedSelection
import matplotlib.pyplot as plt
from coffea.nanoevents.methods import vector
from coffea.nanoevents import NanoEventsFactory

from distributed import Client
import fsspec_xrootd
import dask.multiprocessing
import os

In [ ]:
from msdprocessor import msdProcessor

In [ ]:
client= Client()
client

In [ ]:
qcd_slice = "Pt1000to1400"

In [ ]:
with open("2018_QCD_"+qcd_slice+".json", "rt") as file:
    initial_fileset = json.load(file)

In [ ]:
# print(initial_fileset)

In [ ]:
files={}
for i, j in initial_fileset.items():
    print(f"this is i {i}")
    files[i]={"files":{}}
    for k in j:
        # k = k.replace("cmsxrootd.fnal.gov", "xcache")
        files[i]["files"][k] = "Events"
        print(f"this is k {k}")
    

In [ ]:
preprocessed_available, preprocessed_total = preprocess(
        files,
        step_size=50_000,
        align_clusters=None,
        skip_bad_files=True,
        recalculate_steps=False,
        files_per_batch=1,
        file_exceptions=(OSError,),
        save_form=True,
        uproot_options={},
        step_size_safety_factor=0.5,
    )

In [ ]:
print(preprocessed_available.keys())

In [ ]:
import gzip, pickle, json
output_file = "scaleout_fileset"
with gzip.open(f"{output_file}_available.json.gz", "wt") as file:
    json.dump(preprocessed_available, file, indent=2)
    print(f"Saved available fileset chunks to {output_file}_available.json.gz")
with gzip.open(f"{output_file}_all.json.gz", "wt") as file:
    json.dump(preprocessed_total, file, indent=2)
    print(f"Saved complete fileset chunks to {output_file}_all.json.gz")

In [ ]:
test_preprocessed_files = max_files(preprocessed_available, 10)
test_preprocessed = max_chunks(test_preprocessed_files, 30)

In [ ]:
print(test_preprocessed_files)

In [ ]:
compute, results = apply_to_fileset(data_manipulation=msdProcessor(),    
                            fileset=test_preprocessed_files,
                            schemaclass=PFNanoAODSchema,
                            uproot_options={"allow_read_errors_with_report": (OSError, KeyError)},
                           )

In [ ]:
compute_results=dask.compute(compute)

In [ ]:
print(compute_results)

In [ ]:
util.save(compute_results, "coffea/QCD_"+qcd_slice+".coffea")